In [ ]:
# Usual Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from desisim.survey_release import SurveyRelease

%load_ext autoreload
%autoreload 2

In [ ]:
# Observed data
catdir='/global/cfs/cdirs/desi/science/lya/y1-kp6/iron-tests/catalogs/'
cat_name='QSO_cat_iron_main_dark_healpix_zlya-altbal_zwarn_cut_20230918.fits'
cat_path = os.path.join(catdir,cat_name)

# Generate mock catalog

This function may be introduced intoo a for loop

After making this catalog its path should be passed to quickquasars with the `--from_catalog` flag.

Other flags like `--desi-footprint` are allowed but not needed with this approach.

To include the 440nm dip in the throughput use `--year1-throughput` in quickquasars.

In [ ]:
# Read master catalog and prepare data catalog, takes a little bit of time if data catalg is provided.
# Y5 mocks do not need a catalog path unless you want to use Y1 distributions for magnitudes. 
# Pass invert = True if you want to invert the random numbers generated: 1-random_number.
# This allows generating independent mocks from the same seed.
# TODO: Should probably refactor the class to make it clearer what refers to data and what to mocks.
survey=SurveyRelease(mockteam='london',subversion='v9.0.0',seed=0, 
                     qso_only=True, data_file=cat_path,invert=False)
# Apply redshift distribution
# Note: For Y1 mocks (and probably Y5 too) the target selection redshift distribution from Chaussidon et al. 2022 works better to match QSO targets Iron catalog.
# The option distribution='from_data' should be a better option once I finish implementing it.
survey.apply_redshift_dist(distribution='target_selection',zmin=1.8)
# Apply NPASS geometry:
survey.apply_data_geometry(release='iron') # Pass release = None for Y5 mocks.
# Assign magnitudes 
survey.assign_rband_magnitude(from_data=True) # Pass from_data = False for Y5 mocks. Unless you want to use the Y1 magnitude distributions.
# Assign exposures
survey.assign_exposures(exptime=None) # Pass exptime = 4000 for Y5 mocks.

# Write mock catalog uncomment if needed
#survey.mockcatalog.write('/path/to/output/outputcat_name.fits')

# Quality check on mock catalog

## Redshift distribution

In [ ]:
dndz=plt.hist(survey.mockcatalog['Z'],bins=np.arange(1.8,4,0.1),histtype='step',label='mockcat',lw=2)
plt.hist(survey.data['Z'],bins=np.arange(1.8,4,0.1),histtype='step',label=' data cat',lw=2)
plt.legend(loc='best')
plt.xlabel('Redshift')

## Magnitude distribution

In [ ]:
get_magnitude = lambda f: 22.5-2.5*np.log10(f)
plt.hist(get_magnitude(survey.mockcatalog['FLUX_R']),bins=np.arange(18,24,0.1),histtype='step',label='mockcat',lw=2)
plt.hist(get_magnitude(survey.data['FLUX_R']),bins=np.arange(18,24,0.1),histtype='step',label='data cat',lw=2)
plt.legend(loc='best')
plt.xlabel('r-band Magnitude')

## Footprint

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(survey.mockcatalog['RA'],survey.mockcatalog['DEC'],'.',label='mockcat',ms=0.1)
plt.xlabel('RA')
plt.ylabel('DEC')

## Npasses

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(survey.mockcatalog['RA'],survey.mockcatalog['DEC'],c=survey.mockcatalog['NPASS'],s=0.001,cmap='jet')
plt.colorbar(label='NPASS')
plt.xlabel('RA')
plt.ylabel('DEC')

## Exposure time

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(survey.mockcatalog['RA'],survey.mockcatalog['DEC'],c=survey.mockcatalog['EXPTIME'],s=0.0001,cmap='jet')
plt.colorbar(label='EXPTIME')
plt.xlabel('RA')
plt.ylabel('DEC')